In [52]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [53]:
df = pd.read_csv('hitters.csv')
df.head()

,Unnamed: 0,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,...,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,-Andy Allanson,293,66,1,30,29,14,1,293,66,...,30,29,14,A,E,446,33,20,NaN,A
1,-Alan Ashby,315,81,7,24,38,39,14,3449,835,...,321,414,375,N,W,632,43,10,475.0,N
2,-Alvin Davis,479,130,18,66,72,76,3,1624,457,...,224,266,263,A,W,880,82,14,480.0,A
3,-Andre Dawson,496,141,20,65,78,37,11,5628,1575,...,828,838,354,N,E,200,11,3,500.0,N
4,-Andres Galarraga,321,87,10,39,42,30,2,396,101,...,48,46,33,N,E,805,40,4,91.5,N


In [54]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  322 non-null    object 
 1   AtBat       322 non-null    int64  
 2   Hits        322 non-null    int64  
 3   HmRun       322 non-null    int64  
 4   Runs        322 non-null    int64  
 5   RBI         322 non-null    int64  
 6   Walks       322 non-null    int64  
 7   Years       322 non-null    int64  
 8   CAtBat      322 non-null    int64  
 9   CHits       322 non-null    int64  
 10  CHmRun      322 non-null    int64  
 11  CRuns       322 non-null    int64  
 12  CRBI        322 non-null    int64  
 13  CWalks      322 non-null    int64  
 14  League      322 non-null    object 
 15  Division    322 non-null    object 
 16  PutOuts     322 non-null    int64  
 17  Assists     322 non-null    int64  
 18  Errors      322 non-null    int64  
 19  Salary      263 non-null    f

In [55]:
df['Salary'].fillna(df['Salary'].median(skipna= True),inplace=True )
df.isna().sum()

C:\Users\anuzk\AppData\Local\Temp\ipykernel_15824\1512526769.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Salary'].fillna(df['Salary'].median(skipna= True),inplace=True )


Unnamed: 0    0
AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary        0
NewLeague     0
dtype: int64

In [56]:
dms = pd.get_dummies(df[['League' , 'Division' , 'NewLeague']], drop_first= True)

In [57]:
y=df['Salary']
x_=df.drop(['Unnamed: 0', 'Salary', 'League', 'Division', 'NewLeague'],axis=1).astype('float64')
X=pd.concat([x_,dms[['League_N' , 'Division_W' , 'NewLeague_N']]],axis=1)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [59]:
lasso_model = Lasso().fit(X_train,y_train)
lasso_model.intercept_

C:\Users\anuzk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.137e+07, tolerance: 3.985e+03
  model = cd_fast.enet_coordinate_descent(


305.9538491441522

In [60]:
lasso_model.coef_

array([ -1.73904024,   4.39941502,   4.84466149,   1.00341584,
         0.61153904,   4.1455074 , -13.54040868,  -0.14337423,
         0.20296307,  -0.80657939,   1.18441759,   0.7949082 ,
        -0.68513173,   0.10986752,   0.47257539,  -3.80959837,
        46.26934092, -97.0770164 , -24.3638174 ])

In [66]:
y_predict = lasso_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_predict))


285.8455117649891

In [67]:
from sklearn.metrics import r2_score
r2_score(y_test,y_predict)

0.5264595449608336

In [74]:
from sklearn.linear_model import LassoCV
lasso_cv_model = LassoCV(alphas=np.random.randint(1, 1000, 100), cv=10, max_iter=10000, n_jobs=-1).fit(X_train, y_train)

In [76]:
lasso_cv_model.alpha_

71

In [79]:
lasso_tuned = Lasso().set_params(alpha = 14).fit(X_train , y_train)
y_pred_tuned = lasso_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred_tuned))

C:\Users\anuzk\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.028e+06, tolerance: 3.985e+03
  model = cd_fast.enet_coordinate_descent(


288.85147418989243

In [81]:
pd.Series(lasso_tuned.coef_ , index=X_train.columns)

AtBat          -1.736794
Hits            4.045806
HmRun           1.818052
Runs            1.510614
RBI             1.621120
Walks           4.014419
Years          -7.918470
CAtBat         -0.171018
CHits           0.287205
CHmRun         -0.566881
CRuns           1.153178
CRBI            0.695196
CWalks         -0.655829
PutOuts         0.117337
Assists         0.418028
Errors         -1.984911
League_N        0.000000
Division_W    -37.672288
NewLeague_N     0.000000
dtype: float64